In [ ]:
%pip install okareo
%pip install attrs
%pip install dateutils

In [ ]:
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ.get('OKAREO_API_KEY')
okareo = Okareo(OKAREO_API_KEY)

response = okareo.upload_scenario_set(file_path='./webbizz_10_articles.jsonl', scenario_name="Retrieval Augmented Generation")
print(response)


In [ ]:
import os
from okareo import Okareo
from okareo_api_client.models import ScenarioType

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Use scenario set id from previous step as source for generation
response = okareo.generate_scenarios(
    source_scenario_id="0f06127a-3af6-440f-8f60-5d773a5a3412",
    name="RAG with Vector DB",
    number_examples=2,
    generation_type=ScenarioType.TEXT_REVERSE_QUESTION
)

print(response)

In [ ]:
# perform a test run using a scenario set loaded in the previous cell 
import os
import random
from datetime import datetime
import string
from okareo import Okareo
from okareo.model_under_test import ChromaDb, OpenAIModel
from okareo_api_client.models import TestRunType
from okareo_api_client.models.scenario_set_create import ScenarioSetCreate
from okareo_api_client.models.scenario_set_response import ScenarioSetResponse

# API Key for Okareo runs
OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]

okareo = Okareo(OKAREO_API_KEY)

project_id='d8b4fd20-f958-4386-a000-8bd1c6c1633e'

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(
        name=f"rag_openai_chromadb",
        model=[
            ChromaDb(
                index_name="test-index",
                project_id=project_id,
                top_k=3,
                collection_name="test-collection-openai",
            ),
            OpenAIModel(
                model_id="text-embedding-ada-002",
                model_type="embed",
            ),
        ],
    )

# use the scenario id from the scenario set uploaded in the previous step
scenario_id='d8b4fd20-f958-4386-a000-8bd1c6c1633d'
today_with_time = datetime.now().strftime('%m-%d %H:%M:%S')
test_run_name=f"Retrieval Test Run {today_with_time}"

scenario_set_response = ScenarioSetResponse(
    scenario_id=scenario_id,
    project_id=project_id,
    name="RAG with Chroma DB",    
)

# name must be unique
rnd = "".join(random.choices(string.ascii_letters, k=5))

# generating the scenario set
scenario_set_create = ScenarioSetCreate(
        name=f"rag-chromadb-scenario-set-{rnd}",
        number_examples=1,
        seed_data=[],
        generation_type=ScenarioType.INFORMATION_RETRIEVAL,
)

scenario = okareo.create_scenario_set(scenario_set_create)

test_run_item = model_under_test.run_test_v2(
    scenario=scenario,
    name=test_run_name,
    test_run_type = TestRunType.INFORMATION_RETRIEVAL,
    calculate_metrics=True,
    api_keys={
            "chromadb": "",
            "openai": OPENAI_API_KEY,
    },
)

# display model level metrics for the test run
print(test_run_item.id)
print(test_run_item.model_metrics.to_dict())
